# Desarrollo y datos calculados

### Interpolacion curva de temperaturas
En base a la tabla de pruebas de pendientes por cada $1° C$ hechas a partir de 0.5, 1 y 1.5 lts de carga con la pava voy a interpolar para sacar la curva de pendientes a distintas cargas de agua. Tengo solo los extremos y un punto medio, lo tomo como suficiente para no romper mi pava haciendole tantas pruebas. Uso el polinomio interpolador de Lagrange

Tengo la siguiente tabla de mediciones de mi pava

![img](./mediciones_pava.png)

In [34]:
#primero tabla de diferencias de temperatura con respecto a carga de agua
mediciones_pendiente = [0.5, 1;
                            1, 0.45;
                            1.5, 0.31];
#Después de cuánto se pasa la temperatura a distintos niveles de agua
mediciones_DT = [
                        0.5, 12;
                        1, 8;
                        1.5, 4];

tabla_pendientes = linspace(0.5, 1.5, 100)';
tabla_pendientes(:, 2) = tabla_pendientes(:, 1);
tabla_pendientes(:, 1) = [1:length(tabla_pendientes(:,1))]';

In [35]:
function L = coef_lagrange(tabla, j, x)
    L=1;
    for k=[1:length(tabla)]
        if(k==j)
            continue
        end
    L *= (x - tabla(k, 1)) / (tabla(j, 1) - tabla(k, 1));
    end
end

function valor = lagrange(tabla, x)
    valor=0;;
    for j=[1:length(tabla)]
        valor += tabla(j, 2) * coef_lagrange(tabla, j, x);
    end
end

In [36]:
for x=[1: length(tabla_pendientes(:, 1))]
    tabla_pendientes(x, 3) = lagrange(mediciones_pendiente, tabla_pendientes(x, 2));
    tabla_pendientes(x, 4) = lagrange(mediciones_DT, tabla_pendientes(x, 2));
end

In [37]:
h=figure;
#x=rand(1,10);
#y=1:10;
plot(tabla_pendientes(:,2),tabla_pendientes(:,3),'r');
saveas(h, 'tabla_pendientes','png');
g=figure;
plot(tabla_pendientes(:,2), tabla_pendientes(:,4), 'b')
saveas(g, 'tabla_dif_t', 'png')

La curva roja es la pendiente de cambio de temperatura con respecto a la carga. La azul, los grados que se pasa de la temperatura seteada con respecto a la carga de agua. Ambos están interpolados, yo solo tengo tres datos.

![tabla_pendientes](tabla_pendientes.png)
![tabla curvas temperatura](tabla_dif_t.png)

In [38]:
tabla_pendientes = tabla_pendientes(:, [3:4]);#No necesito ni posicion ni niveles de agua

In [31]:
save tabla_pendientes tabla_pendientes

In [13]:
load tabla_pendientes

## Simulacion comportamiento pava
Al no poder probar esto con una pava real hago la simulación del comportamiento de la pava usando datos estimados y la ecuación de enfriamiento

$$\frac{\partial T}{\partial t} = -k (T - T_m)$$

Vamos a buscar los niveles de k para distintos niveles de agua. Pruebo a mano con la función acercandome con valores para las tres mediciones que ya tengo

In [36]:
f = @(T) -0.0009*(T-400);

f(50)

ans = 0.3150


- Para medio litro $k=-0.003$
- Para un litro $k=-0.0013$
- Para litro y medio $k=-0.0009$

Ahora interpolo para estimar los $k$ con diferencia de un grado. De vuelta con Lagrange

In [39]:
tabla_k = [0.5, -0.003;
        1, -0.0013;
        1.5, -0.0009];
valores_k = [1:10]';
valores_k(:, 2) = linspace(0.5, 1.5, 10)';

In [40]:
for x=[1:10]
    valores_k(x, 3) = lagrange(tabla_k, valores_k(x, 2));
end
valores_k = valores_k(:, 2:3);

Ok, ahora tengo los valores $k$ que determinan la velocidad de calentamiento en base a la cantidad de agua, es lo que después voy a estimar

In [24]:
save valores_k valores_k

Voy a hacer una función `sonda` que setea la temperatura instantanea en base a la temperatura actual y el paso dado. Esta funcion actualiza la variable globale `T_real`. Pongo esta funcion en el codigo principal